# Code Interpreting with Fireworks LLM
This example uses the E2B's [Code Interpreter](https://github.com/e2b-dev/code-interpreter) as a tool for Firefunction-V2 by Fireworks. We ask the Firefunction-V2 LLM to **TBD TBD TBD**
>

### 1. Installations
First, we install Fireworks, OpenAI, and the E2B Code Interpreter SDK.

In [100]:
%pip install --upgrade fireworks-ai openai e2b_code_interpreter

Note: you may need to restart the kernel to use updated packages.


### 2. Imports and API keys
We import packages, get API keys or save them to .env file.

In [101]:
import openai
import json

import os
from dotenv import load_dotenv
load_dotenv()


# TODO: Get your OpenAI API key
FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")

# TODO: Get your E2B API key from https://e2b.dev/docs
E2B_API_KEY = os.getenv("E2B_API_KEY")

from e2b_code_interpreter import CodeInterpreter
code_interpreter = CodeInterpreter(api_key=E2B_API_KEY)

### 3. Model setup, adding tools

Then we set up the model to use.

We describe the functions available to the agent in great detail in JSON Schema. Here, it's just one function, the one that will execute the code generated by the LLM assistant.

**TBD**: Add the fireworks api key as the api_key variable to client.

In [102]:
SYSTEM_PROMPT = """
## your job & context
you are a python data scientist. you are given tasks to complete and you run python code to solve them.
You DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS
- the python code runs in jupyter notebook.
- every time you call `execute_python` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python`.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.
"""


client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = FIREWORKS_API_KEY
)

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Plot a chart visualizing the height distribution of men based on the data you know."}
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_python",
            "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The python code to execute in a single cell.",
                    },
                },
                "required": ["code"],
            },
        },
        },
]


chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/firefunction-v2",
    messages=messages,
    tools=tools,
    temperature=0.1
)
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_A8emjuNqcmyKNk9AhlV9gUYR",
            "function": {
                "arguments": "{\"code\": \"import matplotlib.pyplot as plt\\nimport numpy as np\\n\\n# Assuming a normal distribution with mean 175.5 cm and standard deviation 7.5 cm\\nheights = np.random.normal(175.5, 7.5, 1000)\\n\\nplt.hist(heights, bins=20, edgecolor='black')\\nplt.xlabel('Height (cm)')\\nplt.ylabel('Frequency')\\nplt.title('Height Distribution of Men')\\nplt.show()\"}",
                "name": "execute_python"
            },
            "type": "function",
            "index": 0
        }
    ]
}


### 4. Defining the code interpreter

Now we define the code interpreter.

We use the ```function_call``` to extract the function call information from the chat completion response.

In [103]:
def execute_python(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec.results
  

function_call = chat_completion.choices[0].message.tool_calls[0].function
print(function_call.name)
print(function_call.arguments)
# tool_response = locals()[function_call.name](**json.loads(function_call.arguments))
# print(tool_response)

execute_python
{"code": "import matplotlib.pyplot as plt\nimport numpy as np\n\n# Assuming a normal distribution with mean 175.5 cm and standard deviation 7.5 cm\nheights = np.random.normal(175.5, 7.5, 1000)\n\nplt.hist(heights, bins=20, edgecolor='black')\nplt.xlabel('Height (cm)')\nplt.ylabel('Frequency')\nplt.title('Height Distribution of Men')\nplt.show()"}


### 5. Get agent's response

In [104]:
agent_response = chat_completion.choices[0].message


def chat(e2b_code_interpreter, user_message, base64_image = None, ):
  print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")


  # Append the response from the agent
  messages.append(
      {
          "role": agent_response.role, 
          "content": "",
          "tool_calls": [
              tool_call.model_dump()
              for tool_call in chat_completion.choices[0].message.tool_calls
          ]
      }
  )

  # Append the response from the tool 
  messages.append(
      {
          "role": "function",
          "content": json.dumps(function_call.arguments) # TBD FIX (it was tool_response)
      }
  )

  next_chat_completion = client.chat.completions.create(
      model="accounts/fireworks/models/firefunction-v2",
      messages=messages,
      tools=tools,
      temperature=0.1
  )

  for choice in next_chat_completion.choices:
      if choice.message.tool_calls and len(choice.message.tool_calls) > 0:
        for tool_call in choice.message.tool_calls:
          if tool_call.function.name == "execute_python":
            if "code" in tool_call.function.arguments:
              code = tool_call.function.arguments["code"]
            else:
              code = tool_call.function.arguments
            print("CODE TO RUN")
            print(code)
            code_interpreter_results = execute_python(e2b_code_interpreter, code)
            return code_interpreter_results
      else:
        print(next_chat_completion.choices[0].message.content)



code_interpreter_results = chat(
  code_interpreter,
  messages, # TBD FIX
)
print(code_interpreter_results)
plot1 = code_interpreter_results[0]


print(plot1.raw)
plot1


User Message: [{'role': 'system', 'content': "\n## your job & context\nyou are a python data scientist. you are given tasks to complete and you run python code to solve them.\nYou DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS\n- the python code runs in jupyter notebook.\n- every time you call `execute_python` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python`.\n- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.\n- you have access to the internet and can make api requests.\n- you also have access to the filesystem and can read/write files.\n- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.\n- you can run any python code you want, everything is running in a secure sandbox environment.\n"}, {'role': 'user', 'content': 'Plot a chart visualizing the h

InternalServerError: Error code: 500 - {'error': {'object': 'error', 'type': 'internal_server_error', 'message': 'server had an error while processing your request, please retry again after a brief wait'}}